# Universal Credit

Universal Credit (UC) is a means-tested benefit payment administered by the Department for Work and Pensions (DWP) in the United Kingdom. It was introduced in 2013 to replace several existing means-tested benefits and tax credits for working-age individuals and families. 
Universal Credit was introduced to replace the following six legacy benefits:

* Income-based Jobseeker's Allowance (JSA): Intended for individuals who are actively seeking 
  employment and are on a low income.
* Income-related Employment and Support Allowance (ESA): Designed for individuals who are unable to 
  work due to illness or disability.
* Income Support: Generally for people on a low income who do not qualify for JSA or ESA.
* Child Tax Credit (CTC): Aimed at families or individuals responsible for children.
* Working Tax Credits: Geared toward individuals or families who are on a low income and are working.
* Housing Benefit for working-age claimants: Helps people with the cost of their rent if they are on a 
  low income.

Universal Credit parameters can be found in `policyengine_uk/parameters/gov/dwp/universal_credit` and logic in `policyengine_uk/variables/dwp/universal_credit.py`.

## Legislation

The legal framework for Universal Credit is primarily defined in the legislation provided on the [legislation.gov.uk](https://www.legislation.gov.uk/uksi/2013/376/regulation/22/2021-04-12) website （The Universal Credit Regulations 2013). 

## Universal Credit Rate Changes
Universal Credit consists of various components or elements, each providing financial support for different circumstances. Some of the key components include: 

* Standard allowance 
* Child amounts
* Disabled child additions
* Limited Capability for Work amount
* Limited Capability for Work and Work-Related Activity amount
* Carer amount

The table below shows some of the rates covered by PolicyEngine-UK. 

In [15]:
from policyengine_uk.system import system

parameters = system.parameters

carer_amount = parameters.gov.dwp.universal_credit.elements.carer.amount
child_amount = parameters.gov.dwp.universal_credit.elements.child.amount
disabled_amount = parameters.gov.dwp.universal_credit.elements.disabled.amount
disabled_child_amount = (
    parameters.gov.dwp.universal_credit.elements.child.disabled.amount
)
higher_amount = (
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount
)

elements = [
    carer_amount,
    child_amount,
    disabled_amount,
    disabled_child_amount,
    higher_amount,
]  # [...]

dates = [
    "2016-01-01",
    "2017-01-01",
    "2018-01-01",
    "2019-01-01",
    "2020-01-01",
    "2021-01-01",
    "2022-01-01",
    "2023-01-01",
    "2024-01-01",
]
names = ["Carer", "Child", "Disabled", "Disabled child", "First child"]

import pandas as pd

df = pd.DataFrame()

for date in dates:
    for element, name in zip(elements, names):
        # Append to a dataframe: row = date, column = element, value = amount
        new_row = {"date": date, "element": name, "amount": element(date)}
        # Append row to the dataframe
        df = pd.concat([df, pd.DataFrame([new_row])])


# merge element cells
pivot_df = df.pivot(index="date", columns="element", values="amount")
pivot_df.fillna("")

element      Carer   Child Disabled Disabled child First child
date                                                          
2016-01-01                                                    
2017-01-01  150.39            315.6                           
2018-01-01  151.89           318.76         126.11      277.08
2019-01-01  156.45  231.67   328.32         126.11      277.08
2020-01-01   160.2  231.67    336.2         126.11      277.08
2021-01-01  162.92  235.85   341.92         128.25      281.25
2022-01-01  163.73  237.08   343.63         128.89       282.5
2023-01-01  168.81  244.58   354.28         132.89       290.0
2024-01-01  185.86  269.58   390.06         146.31       315.0

In [16]:
import plotly.express as px
from policyengine_core.charts import format_fig

fig = px.line(
    df,
    title="Universal Credit elements over time",
    x="date",
    y="amount",
    color="element",
).update_layout(
    yaxis_range=[0, 500],
    yaxis_tickformat=",.0f",
    yaxis_tickprefix="£",
    yaxis_title="Amount(£m)",
    xaxis_title="Year",
    legend_title="Element",
)

fig = format_fig(fig)
fig